In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test with Jamie

In [5]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_new_training_week_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
TrainingWeekWithCoaching(typical_week_training_review=Your current training schedule effectively incorporates long runs on Saturdays, which aligns well with your stated preference. Wednesdays appear to be ideal for your workouts as you log higher mileage with a pace of 9m 20s per mile, suggesting potential for speed work on this day. Your lighter runs and potential rest days could be on Sundays, allowing for recovery post long runs., weekly_mileage_target=For the upcoming week, let's aim for a target weekly mileage of around 47 miles. Your longest run should range between 16 to 17 miles, ensuring a gradual increase without overexertion. Over recent weeks, you've successfully handled increasing mileage and can now focus on building endurance for your next marathon. Be attentive to your body's feedback, especially since you've had a consistent traini

In [4]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_mid_week_update_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
athlete_id=98390356 token still valid until 2024-08-30 23:49:42+00:00
TrainingWeekWithPlanning(planning=Planning(weekly_mileage_target=47.0, long_run_distance=16.0, remaining_weekly_mileage=47.0 - 10.04 = 36.96, remaining_weekly_mileage_planning=To achieve a total of 36.96 miles remaining for this week, while keeping the long run at 16.0 miles, we'll adjust the rest of the week's schedule: 

- Wednesday (Speed workout): Keep the planned 7.0 miles. 
- Thursday (Easy run): We'll adjust this to 4.0 miles for better mileage distribution. 
- Friday (Moderate run): Reduce from 6.0 to 5.0 miles to adjust total mileage.
- Saturday (Long run): Keep as planned at 16.0 miles. 
- Sunday (Easy run): Change this from 4.0 miles to 4.96 miles to reach the exact mileage target. 

Calculating total adjusted mileage: 7.0 (Wed) + 4.0 (Thu) + 5.0 (Fri) + 16.0 (Sat) + 4.96 (Sun) = 36.96 miles, precisely matching the required mileage.), tr

# Training week accuracy

In [24]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

Loading 2024-08-28_16-47-43.json
1.97
Loading 2024-08-28_17-27-11.json
1.95
Loading 2024-08-28_17-41-17.json
1.95
Loading 2024-08-29_17-37-07.json
0.75
Loading 2024-08-29_17-45-21.json
0.95
Loading 2024-08-29_17-54-20.json
0.425
Loading 2024-08-29_18-09-32.json
0.475
Loading 2024-08-30_17-57-20.json
0.85
Loading 2024-08-30_18-17-10.json
0.7885
Loading 2024-08-30_18-28-10.json
0.6125
Loading 2024-08-30_19-16-03.json
0.34550000000000003
Loading 2024-08-30_19-21-24.json
0.2
Loading 2024-08-30_19-34-15.json
0.6
Loading 2024-08-30_19-42-10.json
0.4
Loading 2024-08-30_19-48-21.json
0.4


In [25]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)

rlefkee00@gmail.com
target: 30-32
actual: 32.0
error: 0.0
Planning(weekly_mileage_target=30-32 miles, long_run_distance=15.0, remaining_weekly_mileage=32 - 15 = 17 miles or 30 - 15 = 15 miles, remaining_weekly_mileage_planning=To distribute the remaining 17 miles over the 6 other days, let's plan as follows: Two days with easy 5 mile runs (17 - 10 = 7 miles remaining). One moderate run of 5 miles (7 - 5 = 2 miles remaining). One day with a short 2-mile recovery run (2 - 2 = 0 miles remaining). Plus 2 rest days.)
....................................................................................................
voynow99@gmail.com
target: 47-48
actual: 47.0
error: 0.0
Planning(weekly_mileage_target=47-48 miles, long_run_distance=17.0, remaining_weekly_mileage=47 - 17 = 30 miles, remaining_weekly_mileage_planning=To distribute 30 miles over the remaining 6 days: One session of 8 miles (30 - 8 = 22 miles remaining). One session of 7 miles (22 - 7 = 15 miles remaining). One session of 6 mi

# Manual Training Plan Generation

In [67]:
from freezegun import freeze_time
from src.activities import get_activities_df, get_day_of_week_summaries, get_weekly_summaries
from src.auth_manager import get_strava_client
from src.constants import COACH_ROLE
from src.new_training_week import generate_training_week_with_coaching
from src.supabase_client import list_users

@freeze_time("2024-09-01")
def gen_training_week_wrapper(sysmsg_base, strava_client):
    activities_df = get_activities_df(strava_client)
    day_of_week_summaries = get_day_of_week_summaries(activities_df)
    weekly_summaries = get_weekly_summaries(activities_df)
    return generate_training_week_with_coaching(
        sysmsg_base=sysmsg_base,
        weekly_summaries=weekly_summaries,
        day_of_week_summaries=day_of_week_summaries,
    )

# "voynow99@gmail.com", "jaredpalek8@gmail.com", "laurenbrooks01@gmail.com", "raytrip.curtis@gmail.com"
for user in list_users():
    if user.email == "raytrip.curtis@gmail.com":
        print(user.email)
        sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {user.preferences}\n"
        strava_client = get_strava_client(user.athlete_id)
        response = gen_training_week_wrapper(sysmsg_base, strava_client)
        print(response)
        break

raytrip.curtis@gmail.com
athlete_id=58608268 token still valid until 2024-09-03 02:57:23+00:00
TrainingWeekWithCoaching(typical_week_training_review=Your current schedule shows a consistent routine with shorter recovery runs on Monday, and longer efforts on Fridays and Sundays indicating potential long runs. Saturdays, based on your preference, should accommodate your long run for marathon preparation, but current mileage suggests a lighter run. Your speed work on Tuesdays aligns well with your preference, but consider shifting your rest days to Monday or Sunday to ensure recovery after these intensive sessions., weekly_mileage_target=Based on your recent training, you're showing consistent higher mileage with alternating long run distances. For next week, aim for a weekly mileage target of about 65-68 miles to maintain momentum while avoiding burnout. Schedule your long run on Saturday with a distance between 18-20 miles, as you've achieved in Week 32 and 34. Keep focusing on quality 

In [68]:
print(response.total_weekly_mileage)
print(response.weekly_mileage_target)
response.training_week

68.0
Based on your recent training, you're showing consistent higher mileage with alternating long run distances. For next week, aim for a weekly mileage target of about 65-68 miles to maintain momentum while avoiding burnout. Schedule your long run on Saturday with a distance between 18-20 miles, as you've achieved in Week 32 and 34. Keep focusing on quality workouts on Tuesdays, maintaining a pace relevant to your marathon goals, such as around 6m 40s per mile, to further build speed endurance.


[TrainingSession(day=<Day.MON: 'mon'>, session_type=<SessionType.EASY: 'easy run'>, distance=5.0, notes='Easy pace for recovery.'),
 TrainingSession(day=<Day.TUES: 'tues'>, session_type=<SessionType.SPEED: 'speed workout'>, distance=10.0, notes='Intervals at 6m 40s pace, e.g., 6x1mi @ 6m 40s per mile.'),
 TrainingSession(day=<Day.WED: 'wed'>, session_type=<SessionType.MODERATE: 'moderate run'>, distance=9.0, notes='Steady pace, around 7m 30s per mile.'),
 TrainingSession(day=<Day.THURS: 'thurs'>, session_type=<SessionType.MODERATE: 'moderate run'>, distance=9.0, notes='Keep a steady pace, around 7m 30s per mile.'),
 TrainingSession(day=<Day.FRI: 'fri'>, session_type=<SessionType.EASY: 'easy run'>, distance=8.0, notes='Easy pace for recovery, focus on form.'),
 TrainingSession(day=<Day.SAT: 'sat'>, session_type=<SessionType.LONG: 'long run'>, distance=19.0, notes='Long run at a comfortable pace, aim for endurance.'),
 TrainingSession(day=<Day.SUN: 'sun'>, session_type=<SessionType.EASY:

In [54]:
from src.supabase_client import upsert_training_week_with_coaching

upsert_training_week_with_coaching(user.athlete_id, response)

APIResponse(data=[{'id': 69, 'athlete_id': 60813108, 'training_week': '[{"day": "mon", "session_type": "rest day", "distance": 0.0, "notes": "Rest and recovery"}, {"day": "tues", "session_type": "easy run", "distance": 6.0, "notes": "Easy pace"}, {"day": "wed", "session_type": "speed workout", "distance": 8.0, "notes": "3x1 mile @ 7m 30s with 2 minutes rest"}, {"day": "thurs", "session_type": "easy run", "distance": 5.0, "notes": "Easy pace"}, {"day": "fri", "session_type": "rest day", "distance": 0.0, "notes": "Rest and recovery"}, {"day": "sat", "session_type": "long run", "distance": 14.0, "notes": "Steady pace"}, {"day": "sun", "session_type": "easy run", "distance": 5.0, "notes": "Easy pace, recovery run"}]', 'typical_week_training_review': 'Your training schedule aligns well with your marathon goals, with Wednesdays being optimal for workouts as you consistently cover decent mileage at a relatively brisk 8 minutes and 57 seconds per mile pace. Saturdays are effectively utilized f